In [1]:
#by convention, we use these shorter two-letter names
import pysal as ps
import pandas as pd
import numpy as np

# Spatial Data Processing with PySAL & Pandas

PySAL has two simple ways to read in data. But, first, you need to get the path from where your notebook is running on your computer to the place the data is. For example, to find where the notebook is running:

In [2]:
!pwd

/home/ljw/dev/ucgis_workshop_2016/notebooks


PySAL has a command that it uses to get the paths of its example datasets. Let's work with a commonly-used dataset first. 

In [3]:
dbf_path = ps.examples.get_path('NAT.dbf')
print(dbf_path)

/home/ljw/.local/lib/python2.7/site-packages/pysal/examples/nat/NAT.dbf


For the purposes of this part of the workshop, we'll use the `NAT.dbf` example data, and the `usjoin.csv` data. 

In [4]:
csv_path = ps.examples.get_path('usjoin.csv')

### Working with shapefiles

To read in a shapefile, we will need the path to the file.

In [5]:
shp_path = ps.examples.get_path('NAT.shp')
print(shp_path)

/home/ljw/.local/lib/python2.7/site-packages/pysal/examples/nat/NAT.shp


Then, we open the file using the `ps.open` command:

In [6]:
f = ps.open(shp_path)

`f` is what we call a "file handle." That means that it only *points* to the data and provides ways to work with it. By itself, it does not read the whole dataset into memory. To see basic information about the file, we can use a few different methods. 

For instance, the header of the file, which contains most of the metadata about the file:

In [7]:
f.header

{'BBOX Mmax': 0.0,
 'BBOX Mmin': 0.0,
 'BBOX Xmax': -66.9698486328125,
 'BBOX Xmin': -124.7314224243164,
 'BBOX Ymax': 49.371734619140625,
 'BBOX Ymin': 24.95596694946289,
 'BBOX Zmax': 0.0,
 'BBOX Zmin': 3.754550197104843e+72,
 'File Code': 9994,
 'File Length': 731108,
 'Shape Type': 5,
 'Unused0': 0,
 'Unused1': 0,
 'Unused2': 0,
 'Unused3': 0,
 'Unused4': 0,
 'Version': 1000}

To actually read in the shapes from memory, you can use the following commands:

In [8]:
f.by_row(14) #gets the 14th shape from the file

In [9]:
all_polygons = f.read() #reads in all polygons from memory

In [10]:
len(all_polygons)

3085

So, all 3085 polygons have been read in from file. These are stored in PySAL shape objects, which can be used by PySAL and can be converted to other Python shape objects. ]

They typically have a few methods. So, since we've read in polygonal data, we can get some properties about the polygons. Let's just have a look at the first polygon:

In [11]:
all_polygons[0].centroid #the centroid of the first polygon

(-94.90336786329912, 48.771730563701574)

In [12]:
all_polygons[0].area

0.565411079543992

In [13]:
all_polygons[0].perimeter

4.055313773836516

While in the Jupyter Notebook, you can examine what properties an object has by using the tab key.

In [14]:
polygon = all_polygons[0]

In [15]:
polygon. #press tab when the cursor is right after the dot

SyntaxError: invalid syntax (<ipython-input-15-aa03438a2fa8>, line 1)

### Working with Data Tables

When you're working with tables of data, like a `csv` or `dbf`, you can extract your data in the following way. Let's open the dbf file we got the path for above.

In [16]:
f = ps.open(dbf_path)

Just like with the shapefile, we can examine the header of the dbf file

In [17]:
f.header

[u'NAME',
 u'STATE_NAME',
 u'STATE_FIPS',
 u'CNTY_FIPS',
 u'FIPS',
 u'STFIPS',
 u'COFIPS',
 u'FIPSNO',
 u'SOUTH',
 u'HR60',
 u'HR70',
 u'HR80',
 u'HR90',
 u'HC60',
 u'HC70',
 u'HC80',
 u'HC90',
 u'PO60',
 u'PO70',
 u'PO80',
 u'PO90',
 u'RD60',
 u'RD70',
 u'RD80',
 u'RD90',
 u'PS60',
 u'PS70',
 u'PS80',
 u'PS90',
 u'UE60',
 u'UE70',
 u'UE80',
 u'UE90',
 u'DV60',
 u'DV70',
 u'DV80',
 u'DV90',
 u'MA60',
 u'MA70',
 u'MA80',
 u'MA90',
 u'POL60',
 u'POL70',
 u'POL80',
 u'POL90',
 u'DNL60',
 u'DNL70',
 u'DNL80',
 u'DNL90',
 u'MFIL59',
 u'MFIL69',
 u'MFIL79',
 u'MFIL89',
 u'FP59',
 u'FP69',
 u'FP79',
 u'FP89',
 u'BLK60',
 u'BLK70',
 u'BLK80',
 u'BLK90',
 u'GI59',
 u'GI69',
 u'GI79',
 u'GI89',
 u'FH60',
 u'FH70',
 u'FH80',
 u'FH90']

So, the header is a list containing the names of all of the fields we can read. If we were interested in getting the `['NAME', 'STATE_NAME', 'HR90', 'HR80']` fields.

If we just wanted to grab the data of interest, `HR90`, we can use either `by_col` or `by_col_array`, depending on the format we want the resulting data in:

In [18]:
HR90 = f.by_col('HR90')
print(type(HR90).__name__, HR90[0:5])
HR90 = f.by_col_array('HR90')
print(type(HR90).__name__, HR90[0:5])

('list', [0.0, 15.885623511, 6.4624531472, 6.9965017491, 7.4780332772])
('ndarray', array([[  0.        ],
       [ 15.88562351],
       [  6.46245315],
       [  6.99650175],
       [  7.47803328]]))


As you can see, the `by_col` function returns a list of data, with no shape. It can only return one column at a time:

In [19]:
HRs = f.by_col('HR90', 'HR80')

TypeError: __call__() takes exactly 2 arguments (3 given)

This error message is called a "traceback," as you see in the top right, and it usually provides feedback on why the previous command did not execute correctly. Here, you see that one-too-many arguments was provided to `__call__`, which tells us we cannot pass as many arguments as we did to `by_col`.

If you want to read in many columns at once and store them to an array, use `by_col_array`:

In [20]:
HRs = f.by_col_array('HR90', 'HR80')

In [21]:
HRs

array([[  0.        ,   8.85582713],
       [ 15.88562351,  17.20874204],
       [  6.46245315,   3.4507747 ],
       ..., 
       [  4.36732988,   5.2803488 ],
       [  3.72771194,   3.00003   ],
       [  2.04885495,   1.19474313]])

It is best to use `by_col_array` on data of a single type. That is, if you read in a lot of columns, some of them numbers and some of them strings, all columns will get converted to the same datatype:

In [22]:
allcolumns = f.by_col_array(['NAME', 'STATE_NAME', 'HR90', 'HR80'])

In [23]:
allcolumns

array([[u'Lake of the Woods', u'Minnesota', u'0.0', u'8.8558271343'],
       [u'Ferry', u'Washington', u'15.885623511', u'17.208742041'],
       [u'Stevens', u'Washington', u'6.4624531472', u'3.4507746989'],
       ..., 
       [u'York', u'Virginia', u'4.3673298769', u'5.2803488048'],
       [u'Prince William', u'Virginia', u'3.7277119437', u'3.0000300003'],
       [u'Gallatin', u'Montana', u'2.0488549462', u'1.1947431302']], 
      dtype='<U20')

Note that the numerical columns, `HR90` & `HR80` are now considered strings, since they show up with the single tickmarks around them, like `'0.0'`.

These methods work similarly for `.csv` files as well

### Using Pandas with PySAL

A new functionality added to PySAL recently allows you to work with shapefile/dbf pairs using Pandas. This *optional* extension is only turned on if you have Pandas installed. The extension is the `ps.pdio` module:

In [24]:
ps.pdio

<module 'pysal.contrib.pdutilities' from '/home/ljw/.local/lib/python2.7/site-packages/pysal/contrib/pdutilities/__init__.pyc'>

To use it, you can read in shapefile/dbf pairs using the `ps.pdio.read_files` command. 

In [25]:
data_table = ps.pdio.read_files(shp_path)

This reads in *the entire database table* and adds a column to the end, called `geometry`, that stores the geometries read in from the shapefile. 

Now, you can work with it like a standard pandas dataframe.

In [26]:
data_table.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
0,Lake of the Woods,Minnesota,27,077,27077,27,77,27077,0,0.000000,...,0.024534,0.285235,0.372336,0.342104,0.336455,11.279621,5.4,5.663881,9.515860,<pysal.cg.shapes.Polygon object at 0x7fb72821e...
1,Ferry,Washington,53,019,53019,53,19,53019,0,0.000000,...,0.317712,0.256158,0.360665,0.361928,0.360640,10.053476,2.6,10.079576,11.397059,<pysal.cg.shapes.Polygon object at 0x7fb727783...
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7fb727783...
3,Okanogan,Washington,53,047,53047,53,47,53047,0,2.612330,...,0.155922,0.258540,0.371218,0.381240,0.394519,9.039900,8.1,10.084926,12.840340,<pysal.cg.shapes.Polygon object at 0x7fb727783...
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7fb727783...


To get the count of all observations within each state:

The `read_files` function only works on shapefile/dbf pairs. If you need to read in data using CSVs, use pandas directly:

In [27]:
usjoin = pd.read_csv(csv_path)
#usjoin = ps.pdio.read_files(usjoin) #will not work, not a shp/dbf pair

The nice thing about working with pandas dataframes is that they have very powerful baked-in support for relational-style queries. By this, I mean that it is very easy to find things like:

The number of counties in each state:

In [28]:
data_table.groupby("STATE_NAME").size()

STATE_NAME
Alabama                  67
Arizona                  14
Arkansas                 75
California               58
Colorado                 63
Connecticut               8
Delaware                  3
District of Columbia      1
Florida                  67
Georgia                 159
Idaho                    44
Illinois                102
Indiana                  92
Iowa                     99
Kansas                  105
Kentucky                120
Louisiana                64
Maine                    16
Maryland                 24
Massachusetts            12
Michigan                 83
Minnesota                87
Mississippi              82
Missouri                115
Montana                  55
Nebraska                 93
Nevada                   17
New Hampshire            10
New Jersey               21
New Mexico               32
New York                 58
North Carolina          100
North Dakota             53
Ohio                     88
Oklahoma                 77
Oregon   

Or, to get the rows of the table that are in Arizona, we can use the `query` function of the dataframe:

In [29]:
data_table.query('STATE_NAME == "Arizona"')

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
1707,Navajo,Arizona,04,017,04017,4,17,4017,0,5.263989,...,0.905251,0.366863,0.414135,0.401999,0.445299,13.146998,12.1,13.762783,18.033782,<pysal.cg.shapes.Polygon object at 0x7fb726f31...
1708,Coconino,Arizona,04,005,04005,4,5,4005,0,3.185449,...,1.469081,0.301222,0.377785,0.381655,0.403188,9.475171,8.5,11.181563,15.267643,<pysal.cg.shapes.Polygon object at 0x7fb726f31...
1722,Mohave,Arizona,04,015,04015,4,15,4015,0,0.000000,...,0.324075,0.279339,0.347150,0.375790,0.374383,11.508554,4.8,7.018268,9.214294,<pysal.cg.shapes.Polygon object at 0x7fb726ece...
1726,Apache,Arizona,04,001,04001,4,1,4001,0,10.951223,...,0.162361,0.395913,0.450552,0.431013,0.489132,15.014738,14.6,18.727548,22.933635,<pysal.cg.shapes.Polygon object at 0x7fb726ece...
2002,Yavapai,Arizona,04,025,04025,4,25,4025,0,3.458771,...,0.298011,0.289509,0.378195,0.376313,0.384089,9.930032,8.6,7.516372,9.483521,<pysal.cg.shapes.Polygon object at 0x7fb726e34...
2182,Gila,Arizona,04,007,04007,4,7,4007,0,6.473749,...,0.246171,0.265294,0.337519,0.353848,0.386976,10.470261,8.1,9.934237,11.706102,<pysal.cg.shapes.Polygon object at 0x7fb726db7...
2262,Maricopa,Arizona,04,013,04013,4,13,4013,0,6.179259,...,3.499221,0.277828,0.352374,0.366015,0.372756,10.642382,9.8,11.857260,14.404902,<pysal.cg.shapes.Polygon object at 0x7fb726d0a...
2311,Greenlee,Arizona,04,011,04011,4,11,4011,0,2.896284,...,0.349650,0.177691,0.257158,0.283518,0.337256,9.806115,6.7,5.295110,10.453284,<pysal.cg.shapes.Polygon object at 0x7fb726d48...
2326,Graham,Arizona,04,009,04009,4,9,4009,0,4.746648,...,1.890487,0.310256,0.362926,0.383554,0.408379,11.979335,10.1,11.961367,16.129032,<pysal.cg.shapes.Polygon object at 0x7fb726d48...
2353,Pinal,Arizona,04,021,04021,4,21,4021,0,13.828390,...,3.134586,0.304294,0.369974,0.361193,0.400130,10.822965,8.8,10.341699,15.304144,<pysal.cg.shapes.Polygon object at 0x7fb726ce8...


Behind the scenes, what this does is compare the `STATE_NAME` column to `'Arizona'`:

In [30]:
data_table.STATE_NAME == 'Arizona'

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3055    False
3056    False
3057    False
3058    False
3059    False
3060    False
3061    False
3062    False
3063    False
3064    False
3065    False
3066    False
3067    False
3068    False
3069    False
3070    False
3071    False
3072    False
3073    False
3074    False
3075    False
3076    False
3077    False
3078    False
3079    False
3080     True
3081    False
3082    False
3083    False
3084    False
Name: STATE_NAME, dtype: bool

And then uses that to filter out rows where the condition is true:

In [31]:
data_table[data_table.STATE_NAME == 'Arizona']

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
1707,Navajo,Arizona,04,017,04017,4,17,4017,0,5.263989,...,0.905251,0.366863,0.414135,0.401999,0.445299,13.146998,12.1,13.762783,18.033782,<pysal.cg.shapes.Polygon object at 0x7fb726f31...
1708,Coconino,Arizona,04,005,04005,4,5,4005,0,3.185449,...,1.469081,0.301222,0.377785,0.381655,0.403188,9.475171,8.5,11.181563,15.267643,<pysal.cg.shapes.Polygon object at 0x7fb726f31...
1722,Mohave,Arizona,04,015,04015,4,15,4015,0,0.000000,...,0.324075,0.279339,0.347150,0.375790,0.374383,11.508554,4.8,7.018268,9.214294,<pysal.cg.shapes.Polygon object at 0x7fb726ece...
1726,Apache,Arizona,04,001,04001,4,1,4001,0,10.951223,...,0.162361,0.395913,0.450552,0.431013,0.489132,15.014738,14.6,18.727548,22.933635,<pysal.cg.shapes.Polygon object at 0x7fb726ece...
2002,Yavapai,Arizona,04,025,04025,4,25,4025,0,3.458771,...,0.298011,0.289509,0.378195,0.376313,0.384089,9.930032,8.6,7.516372,9.483521,<pysal.cg.shapes.Polygon object at 0x7fb726e34...
2182,Gila,Arizona,04,007,04007,4,7,4007,0,6.473749,...,0.246171,0.265294,0.337519,0.353848,0.386976,10.470261,8.1,9.934237,11.706102,<pysal.cg.shapes.Polygon object at 0x7fb726db7...
2262,Maricopa,Arizona,04,013,04013,4,13,4013,0,6.179259,...,3.499221,0.277828,0.352374,0.366015,0.372756,10.642382,9.8,11.857260,14.404902,<pysal.cg.shapes.Polygon object at 0x7fb726d0a...
2311,Greenlee,Arizona,04,011,04011,4,11,4011,0,2.896284,...,0.349650,0.177691,0.257158,0.283518,0.337256,9.806115,6.7,5.295110,10.453284,<pysal.cg.shapes.Polygon object at 0x7fb726d48...
2326,Graham,Arizona,04,009,04009,4,9,4009,0,4.746648,...,1.890487,0.310256,0.362926,0.383554,0.408379,11.979335,10.1,11.961367,16.129032,<pysal.cg.shapes.Polygon object at 0x7fb726d48...
2353,Pinal,Arizona,04,021,04021,4,21,4021,0,13.828390,...,3.134586,0.304294,0.369974,0.361193,0.400130,10.822965,8.8,10.341699,15.304144,<pysal.cg.shapes.Polygon object at 0x7fb726ce8...


We might need this behind the scenes knowledge when we want to chain together conditions, or when we need to do spatial queries. 

This is because spatial queries are somewhat more complex. Let's say, for example, we want all of the counties in the US to the West of `-121` longitude. We need a way to express that question. Ideally, we want something like:

```
SELECT
        *
FROM
        data_table
WHERE
        x_centroid < -121
```

So, let's refer to an arbitrary polygon in the the dataframe's geometry column as `poly`. The centroid of a PySAL polygon is stored as an `(X,Y)` pair, so the longidtude is the first element of the pair, `poly.centroid[0]`. 

Then, applying this condition to each geometry, we get the same kind of filter we used above to grab only counties in Arizona:

In [32]:
data_table.geometry.apply(lambda poly: poly.centroid[0] < -121)

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27       True
28      False
29      False
        ...  
3055    False
3056    False
3057    False
3058    False
3059    False
3060    False
3061    False
3062    False
3063    False
3064    False
3065    False
3066    False
3067    False
3068    False
3069    False
3070    False
3071    False
3072    False
3073    False
3074    False
3075    False
3076    False
3077    False
3078    False
3079    False
3080    False
3081    False
3082    False
3083    False
3084    False
Name: geometry, dtype: bool

If we use this as a filter on the table, we can get only the rows that match that condition, just like we did for the `STATE_NAME` query:

In [33]:
data_table[data_table.geometry.apply(lambda x: x.centroid[0] < -121)]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
27,Whatcom,Washington,53,073,53073,53,73,53073,0,1.422131,...,0.508687,0.247630,0.346935,0.369436,0.358418,9.174415,7.1,9.718054,11.135022,<pysal.cg.shapes.Polygon object at 0x7fb72741e...
31,Skagit,Washington,53,057,53057,53,57,53057,0,2.596560,...,0.351958,0.239346,0.344830,0.364623,0.362265,8.611518,7.9,10.480031,11.382484,<pysal.cg.shapes.Polygon object at 0x7fb72741e...
44,Clallam,Washington,53,009,53009,53,9,53009,0,3.330891,...,0.568504,0.240573,0.349320,0.361619,0.366854,8.788882,6.5,9.660900,12.281690,<pysal.cg.shapes.Polygon object at 0x7fb72741e...
47,Snohomish,Washington,53,061,53061,53,61,53061,0,2.129319,...,1.023748,0.234133,0.300980,0.331988,0.325067,8.244432,7.4,10.701071,12.202467,<pysal.cg.shapes.Polygon object at 0x7fb72741e...
48,Island,Washington,53,029,53029,53,29,53029,0,0.000000,...,2.415483,0.252990,0.357682,0.369762,0.350930,8.854708,6.8,8.505577,8.509372,<pysal.cg.shapes.Polygon object at 0x7fb72741e...
57,Jefferson,Washington,53,031,53031,53,31,53031,0,0.000000,...,0.416956,0.229855,0.326232,0.379629,0.381017,8.978723,6.0,9.088992,10.860725,<pysal.cg.shapes.Polygon object at 0x7fb72743f...
71,Kitsap,Washington,53,035,53035,53,35,53035,0,0.791991,...,2.691706,0.241070,0.315654,0.338109,0.347093,9.499651,8.1,9.760890,11.430652,<pysal.cg.shapes.Polygon object at 0x7fb72743f...
80,King,Washington,53,033,53033,53,33,53033,0,3.351108,...,5.061238,0.250251,0.317439,0.346038,0.335487,10.452660,9.8,12.742441,14.089579,<pysal.cg.shapes.Polygon object at 0x7fb72743f...
85,Mason,Washington,53,045,53045,53,45,53045,0,0.000000,...,0.865914,0.243041,0.342107,0.357727,0.361988,7.596109,6.1,9.091962,9.157372,<pysal.cg.shapes.Polygon object at 0x7fb72743f...
92,Grays Harbor,Washington,53,027,53027,53,27,53027,0,1.224028,...,0.185430,0.253515,0.338063,0.355237,0.379909,9.881310,8.7,10.305788,13.887620,<pysal.cg.shapes.Polygon object at 0x7fb7273df...


This works on any type of spatial query. For instance, if we wanted to find all of the counties with large areas:

In [34]:
data_table[data_table.geometry.apply(lambda x: x.area > 4 )]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
603,Elko,Nevada,32,007,32007,32,7,32007,0,8.325701,...,0.793319,0.277065,0.317528,0.368389,0.325195,9.740933,8.0,10.281598,11.141615,<pysal.cg.shapes.Polygon object at 0x7fb727275...
1207,Nye,Nevada,32,023,32023,32,23,32023,0,22.862369,...,1.636578,0.241638,0.311393,0.329568,0.346481,10.174717,4.6,5.879900,8.084471,<pysal.cg.shapes.Polygon object at 0x7fb7270a1...
1708,Coconino,Arizona,04,005,04005,4,5,4005,0,3.185449,...,1.469081,0.301222,0.377785,0.381655,0.403188,9.475171,8.5,11.181563,15.267643,<pysal.cg.shapes.Polygon object at 0x7fb726f31...
1957,San Bernardino,California,06,071,06071,6,71,6071,0,3.243373,...,8.103188,0.259038,0.350927,0.362389,0.372659,9.857519,10.0,12.999831,15.403925,<pysal.cg.shapes.Polygon object at 0x7fb726e7d...


### Moving in and out of the dataframe

Most things in PySAL will be explicit about what type their input should be. Most of the time, PySAL functions require either lists or arrays. This is why the file-handler methods are the default IO method in PySAL: the rest of the computational tools are built around their datatypes. 

However, it is very easy to get the correct datatype from Pandas using the `values` and `tolist` commands. 

`tolist()` will convert its entries to a list. But, it can only be called on individual columns (called `Series` in `pandas` documentation)

So, to turn the `NAME` column into a list:

In [41]:
data_table.NAME.tolist()

[u'Lake of the Woods',
 u'Ferry',
 u'Stevens',
 u'Okanogan',
 u'Pend Oreille',
 u'Boundary',
 u'Lincoln',
 u'Flathead',
 u'Glacier',
 u'Toole',
 u'Liberty',
 u'Hill',
 u'Sheridan',
 u'Divide',
 u'Burke',
 u'Renville',
 u'Bottineau',
 u'Rolette',
 u'Towner',
 u'Cavalier',
 u'Pembina',
 u'Kittson',
 u'Roseau',
 u'Blaine',
 u'Phillips',
 u'Valley',
 u'Daniels',
 u'Whatcom',
 u'Bonner',
 u'Ward',
 u'Koochiching',
 u'Skagit',
 u'Williams',
 u'McHenry',
 u'St. Louis',
 u'Roosevelt',
 u'Mountrial',
 u'Marshall',
 u'Ramsey',
 u'Walsh',
 u'Beltrami',
 u'Pierce',
 u'Chelan',
 u'Pondera',
 u'Clallam',
 u'Benson',
 u'Chouteau',
 u'Snohomish',
 u'Island',
 u'Sanders',
 u'Lake',
 u'Nelson',
 u'Grand Forks',
 u'Polk',
 u'Pennington',
 u'Douglas',
 u'McKenzie',
 u'Jefferson',
 u'Richland',
 u'Teton',
 u'McCone',
 u'Shoshone',
 u'Spokane',
 u'Lake',
 u'Clearwater',
 u'Kootenai',
 u'Garfield',
 u'Red Lake',
 u'Grant',
 u'Lincoln',
 u'Lewis and Clark',
 u'Kitsap',
 u'Itasca',
 u'Sheridan',
 u'Wells',
 u'

To extract many columns, you must select the columns you want and call their `.values` attribute. 

If we were interested in grabbing all of the `HR` variables in the dataframe, we could first select those column names:

In [37]:
HRs = [col for col in data_table.columns if col.startswith('HR')]

We can use this to focus only on the columns we want:

In [39]:
data_table[HRs]

,HR60,HR70,HR80,HR90
0,0.000000,0.000000,8.855827,0.000000
1,0.000000,0.000000,17.208742,15.885624
2,1.863863,1.915158,3.450775,6.462453
3,2.612330,1.288643,3.263814,6.996502
4,0.000000,0.000000,7.770008,7.478033
5,0.000000,0.000000,4.573101,4.000640
6,7.976390,5.536179,5.633168,5.720497
7,1.011173,1.689475,4.490115,2.814460
8,11.529039,9.273857,28.227324,5.500096
9,0.000000,5.708740,0.000000,6.605892


With this, calling `.values` gives an array containing all of the entries in this subset of the table:

In [40]:
data_table[HRs].values

array([[  0.        ,   0.        ,   8.85582713,   0.        ],
       [  0.        ,   0.        ,  17.20874204,  15.88562351],
       [  1.86386342,   1.91515848,   3.4507747 ,   6.46245315],
       ..., 
       [  1.5444254 ,   6.02355209,   5.2803488 ,   4.36732988],
       [  9.30282008,   1.80014761,   3.00003   ,   3.72771194],
       [  3.39616234,   2.28487867,   1.19474313,   2.04885495]])

Using the PySAL pdio tools means that if you're comfortable with working in Pandas, you can continue to do so. 

If you're more comfortable using Numpy or raw Python to do your data processing, PySAL's IO tools naturally support this. 